# Assignment 5: Fake News Detection using LSTM + BiLSTM
## **CSS432 Natural Language Processing and Information Retrieval**
### Section 1

---

## **Objective:**


The goal of this project is to create and assess LSTM and BiLSTM models specifically for identifying fake news. This endeavor will provide students with hands-on knowledge in the fields of natural language processing (NLP) and advanced learning strategies, encompassing areas such as text preparation, crafting model frameworks, adjusting hyperparameters, and evaluating model performance. By conducting experiments and analyzing results, students will investigate how various model configurations and parameters impact the accuracy of fake news detection, enhancing their grasp of applying deep learning techniques to text categorization challenges.



---

## **Download Dataset**

> [download link](https://gitlab.com/atlonxp/siit-nlp/-/raw/main/dl-rnn/fake-new-dataset.zip?ref_type=heads&inline=false )



In [ ]:
# Download
!curl "https://gitlab.com/atlonxp/siit-nlp/-/raw/main/dl-rnn/fake-new-dataset.zip?ref_type=heads&inline=false" --output "fake-news.zip"

# Extract
import zipfile
with zipfile.ZipFile('fake-news.zip','r') as zip_ref:
  zip_ref.extractall()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.8M  100 90.8M    0     0  95.3M      0 --:--:-- --:--:-- --:--:-- 95.3M




---


# LSTM

## Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv

## Preprocessing

Convert csv file into list

In [ ]:
filename = "fake-news-dataset.csv"
dataset = open(filename, 'r')
dataset_reader = csv.reader(dataset)

# Set new field size limit
csv.field_size_limit(1000000)

# Convert to list
dataset_list = list(dataset_reader)

#Remove index column
dataset_list = [row[1:] for row in dataset_list]
print(dataset_list[0])

['title', 'text', 'label']


See the dataset

In [ ]:
df = pd.DataFrame(dataset_list[1:6], columns=dataset_list[0])
df.style

Since the text is very long, only the title will be used.

In [ ]:
#Remove text column
dataset_list = [[row[0]] + [row[-1]] for row in dataset_list]
df = pd.DataFrame(dataset_list[1:6], columns=dataset_list[0])
df.style

,title,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO],1
1,,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO],1
3,"Bobby Jindal, raised Hindu, uses story of Christian conversion to woo evangelicals for potential 2016 bid",0
4,SATAN 2: Russia unvelis an image of its terrifying new ‘SUPERNUKE’ – Western world takes notice,1


Remove entry with empty title

In [ ]:
#Remove empty title
dataset_list = [row for row in dataset_list if row[0]]
df = pd.DataFrame(dataset_list[1:6], columns=dataset_list[0])
df.style

,title,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO],1
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO],1
2,"Bobby Jindal, raised Hindu, uses story of Christian conversion to woo evangelicals for potential 2016 bid",0
3,SATAN 2: Russia unvelis an image of its terrifying new ‘SUPERNUKE’ – Western world takes notice,1
4,About Time! Christian Group Sues Amazon and SPLC for Designation as Hate Group,1


In [ ]:
print(len(dataset_list))

71577


Remove entry with empty label

In [ ]:
dataset_list = [row for row in dataset_list if row[1].isnumeric()]
print(len(dataset_list))

71576


Preprocess Text

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Adapted from assignment 1
def process_title(title):
    """Process tweet function.
    Input:
        title: a string containing a title
    Output:
        title_clean: a string of cleaned title

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # only take alphabets
    title = re.sub(r'[^a-zA-Z]', ' ', title)
    # lower case
    title = title.lower()
    # tokenize
    title_token = word_tokenize(title, language='english', preserve_line=False)

    title_clean = ""
    for word in title_token:
        if (word not in stopwords_english): # remove stopwords
            stem_word = stemmer.stem(word)  # stemming word
            title_clean =  title_clean + " " + stem_word

    return title_clean

Test preprocess_title

In [ ]:
print(process_title(dataset_list[1][0]))
print("\n" + dataset_list[1][0])

 unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video

UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO]


Preprocess the dataset

In [ ]:
for row in dataset_list:
  row[0] = process_title(row[0])

See Preprocessed title

In [ ]:
df = pd.DataFrame(dataset_list[:20], columns=["title", "label"])
df.style

,title,label
0,law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video,1
1,unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video,1
2,bobbi jindal rais hindu use stori christian convers woo evangel potenti bid,0
3,satan russia unv imag terrifi new supernuk western world take notic,1
4,time christian group sue amazon splc design hate group,1
5,dr ben carson target ir never audit spoke nation prayer breakfast,1
6,hous intel chair trump russia fake stori evid anyth video,1
7,sport bar owner ban nfl game show true american sport like speak rural america video,1
8,latest pipelin leak underscor danger dakota access pipelin,1
9,gop senat smack punchabl alt right nazi internet,1




---
## Define model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

Split into corpus and labels

In [ ]:
corpus = [row[0] for row in dataset_list]
label = [int(row[1]) for row in dataset_list]

In [ ]:
for i in range(20):
  print(corpus[i], label[i])

 law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video 1
 unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video 1
 bobbi jindal rais hindu use stori christian convers woo evangel potenti bid 0
 satan russia unv imag terrifi new supernuk western world take notic 1
 time christian group sue amazon splc design hate group 1
 dr ben carson target ir never audit spoke nation prayer breakfast 1
 hous intel chair trump russia fake stori evid anyth video 1
 sport bar owner ban nfl game show true american sport like speak rural america video 1
 latest pipelin leak underscor danger dakota access pipelin 1
 gop senat smack punchabl alt right nazi internet 1
 may brexit offer would hurt cost eu citizen eu parliament 0
 schumer call trump appoint offici overse puerto rico relief 0
 watch hilari ad call question health age clinton crime famili boss 1
 chang expect espn polit agenda despit huge subscrib declin breitbart 0
 billionair 

Split into training and test set.

In [ ]:
dataset_size = len(label)
split_size = int(dataset_size * 0.8)
x_train = np.array(corpus[:split_size])
y_train =  np.array(label[:split_size])
x_test =  np.array(corpus[split_size:])
y_test =  np.array(label[split_size:])

Check label distribution

In [ ]:
train_sum = 0
test_sum = 0
for i in y_train:
  train_sum += i
for i in y_test:
  test_sum += i

print("training set:", train_sum/split_size)
print("test set:", test_sum/(dataset_size-split_size))

training set: 0.5104785190359763
test set: 0.5111763062307907


Text Vectorization

In [ ]:
text_vectorizer = TextVectorization(output_mode="int")
text_vectorizer.adapt(x_train)

In [ ]:
for i in range(5):
  print(text_vectorizer(x_train[200+i]).numpy())
  print(x_train[200+i])

[ 191  390 3204  316  650 3213 6900  898  426   39  417]
 famili arm robberi suspect outrag pizza hut employe shot kill son
[  59    2  553 2769    2   29  185  409  428 2129    3]
 anti trump radic taunt trump support isi flag photo behead video
[ 279 3819 4414  704   85  146   32  683  166  277    3]
 wow atlanta gym owner ban cop make apolog polici sign video
[ 699   12  340 2978  277  152 1112 3911    4    6    5]
 turkish presid return istanbul sign militari coup falter new york time
[  112  4743   650  4826  4893    18 13611   678  6562  1001   421 12036]
 comment sjw outrag leonardo dicaprio white rumi role unfound iranian explain ztech


Define sequential model

User binary_crossentropy because output is 0 or 1

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=32))
model.add(LSTM(128, input_shape=(1,32), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          583808    
                                                                 
 lstm_6 (LSTM)               (None, None, 128)         82432     
                                                                 
 dropout_6 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_7 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 797953 (3.04 MB)
Trainable params: 79795



---

## Define checkpoint

In [ ]:
filepath = "LSTM-model2-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
checkpoint_LSTM = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list_LSTM = [checkpoint_LSTM]



---
## Fit model


Text vectorize training and testing set

In [ ]:
x_train_v = [text_vectorizer(text) for text in x_train]
x_test_v = [text_vectorizer(text) for text in x_test]

See vectorized titles

In [ ]:
for i in range(20):
  print(x_train_v[1000+i])

tf.Tensor([ 251 6666  373 2579 4714  692  421 1086 1380  542], shape=(10,), dtype=int64)
tf.Tensor([ 182 1326  202 2361  243  291   75], shape=(7,), dtype=int64)
tf.Tensor([ 423   57   32 1798 2155 7956], shape=(6,), dtype=int64)
tf.Tensor([  164    72  3052 11140   514  9335], shape=(6,), dtype=int64)
tf.Tensor([ 352 1391 3576  877  309 3352 7119   22 1842 7120 2000  133   14], shape=(13,), dtype=int64)
tf.Tensor([1933  154  139 1066 1423  400  211  653], shape=(8,), dtype=int64)
tf.Tensor([ 221  535  775 2078 2806    2  379 1140  768], shape=(9,), dtype=int64)
tf.Tensor([    2   179    41   564  2793    96 16854    44  2331], shape=(9,), dtype=int64)
tf.Tensor([  18   13  893 1391 1373  388    2   44  427  300    2 3206], shape=(12,), dtype=int64)
tf.Tensor([   2    8   63  791  111  118 2729  264], shape=(8,), dtype=int64)
tf.Tensor([ 25 643 966 177 248  72  70], shape=(7,), dtype=int64)
tf.Tensor([ 400    8  123 2641  340  653  137], shape=(7,), dtype=int64)
tf.Tensor([  41 1629 11

In [ ]:
with open('x_train_v.csv', 'w') as f:
    write = csv.writer(f)
with open('x_test_v.csv', 'w') as f:
    write = csv.writer(f)

Pad sequence to equal length

In [ ]:
x_train_vp = pad_sequences(x_train_v, padding="post")
x_test_vp = pad_sequences(x_test_v, padding="post")

Fit model

In [ ]:
model.fit(x_train_vp, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks_list_LSTM)

Epoch 1/10
716/716 [==============================] - ETA: 0s - loss: 0.4183 - accuracy: 0.7669
Epoch 1: loss improved from inf to 0.41825, saving model to LSTM-model2-weights-improvement-01-0.4183-bigger.keras
716/716 [==============================] - 205s 278ms/step - loss: 0.4183 - accuracy: 0.7669 - val_loss: 0.2706 - val_accuracy: 0.8901
Epoch 2/10
716/716 [==============================] - ETA: 0s - loss: 0.2188 - accuracy: 0.9161
Epoch 2: loss improved from 0.41825 to 0.21879, saving model to LSTM-model2-weights-improvement-02-0.2188-bigger.keras
716/716 [==============================] - 179s 251ms/step - loss: 0.2188 - accuracy: 0.9161 - val_loss: 0.2533 - val_accuracy: 0.8998
Epoch 3/10
716/716 [==============================] - ETA: 0s - loss: 0.1830 - accuracy: 0.9309
Epoch 3: loss improved from 0.21879 to 0.18299, saving model to LSTM-model2-weights-improvement-03-0.1830-bigger.keras
716/716 [==============================] - 190s 265ms/step - loss: 0.1830 - accuracy: 0.9



---

## Test the model and metrics such as accuracy, precision, recall, and F1-score

Testing the model with least training loss VS least validation loss

In [ ]:
model_loss = Sequential()
model_loss.add(Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=32))
model_loss.add(LSTM(128, input_shape=(1,32), return_sequences=True))
model_loss.add(Dropout(0.2))
model_loss.add(LSTM(128))
model_loss.add(Dropout(0.2))
model_loss.add(Dense(1, activation='sigmoid'))
model_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_loss.summary()

model_val_loss = Sequential()
model_val_loss.add(Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=32))
model_val_loss.add(LSTM(128, input_shape=(1,32), return_sequences=True))
model_val_loss.add(Dropout(0.2))
model_val_loss.add(LSTM(128))
model_val_loss.add(Dropout(0.2))
model_val_loss.add(Dense(1, activation='sigmoid'))
model_val_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_val_loss.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          583808    
                                                                 
 lstm_8 (LSTM)               (None, None, 128)         82432     
                                                                 
 dropout_8 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 797953 (3.04 MB)
Trainable params: 79795

In [ ]:
#least training loss - Epoch 10
filename_loss = "/content/LSTM-model2-weights-improvement-10-0.0668-bigger.keras"
model_loss.load_weights(filename_loss)
model_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#least validation loss - Epoch 3
filename_val_loss = "/content/LSTM-model2-weights-improvement-03-0.1830-bigger.keras"
model_val_loss.load_weights(filename_val_loss)
model_val_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Predict on test set

In [ ]:
y_predict_loss = model_loss.predict(x_test_vp)
y_predict_val_loss = model_val_loss.predict(x_test_vp)

448/448 [==============================] - 18s 37ms/step


In [ ]:
y_predict_loss = y_predict_loss.round()
y_predict_val_loss = y_predict_val_loss.round()

Metrics

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

#y_test is label
#y_predict is prediction
print("Least training loss:\n")
print("Accuracy:", accuracy_score(y_test, y_predict_loss))
print( classification_report(y_test, y_predict_loss))
print("\nLeast validation loss:\n")
print("Accuracy:", accuracy_score(y_test, y_predict_val_loss))
print( classification_report(y_test, y_predict_val_loss))

Least training loss:

Accuracy: 0.8640681754680078
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      6998
           1       0.86      0.87      0.87      7318

    accuracy                           0.86     14316
   macro avg       0.86      0.86      0.86     14316
weighted avg       0.86      0.86      0.86     14316


Least validation loss:

Accuracy: 0.877689298686784
              precision    recall  f1-score   support

           0       0.92      0.82      0.87      6998
           1       0.84      0.93      0.89      7318

    accuracy                           0.88     14316
   macro avg       0.88      0.88      0.88     14316
weighted avg       0.88      0.88      0.88     14316





---
#BiLSTM



---
## Define model


In [ ]:
from tensorflow.keras.layers import Bidirectional

Define sequential model

Using two Bidirectional layers

In [ ]:
model_bi = Sequential()
model_bi.add(Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=32))
model_bi.add(Bidirectional(LSTM(64, input_shape=(1,32), return_sequences=True)))
model_bi.add(Dropout(0.2))
model_bi.add(Bidirectional(LSTM(64)))
model_bi.add(Dropout(0.2))
model_bi.add(Dense(1, activation='sigmoid'))
model_bi.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bi.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 32)          583808    
                                                                 
 bidirectional_2 (Bidirecti  (None, None, 128)         49664     
 onal)                                                           
                                                                 
 dropout_22 (Dropout)        (None, None, 128)         0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dropout_23 (Dropout)        (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 1)               



---
## Define checkpoint

In [ ]:
filepath_BiLSTM = "BiLSTM-model2-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
checkpoint_BiLSTM = ModelCheckpoint(filepath_BiLSTM, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list_BiLSTM = [checkpoint_BiLSTM]



---
## Fit model


In [ ]:
model_bi.fit(x_train_vp, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks_list_BiLSTM)

Epoch 1/10
716/716 [==============================] - ETA: 0s - loss: 0.2839 - accuracy: 0.8766
Epoch 1: loss improved from inf to 0.28393, saving model to BiLSTM-model2-weights-improvement-01-0.2839-bigger.keras
716/716 [==============================] - 220s 229ms/step - loss: 0.2839 - accuracy: 0.8766 - val_loss: 0.2306 - val_accuracy: 0.9087
Epoch 2/10
716/716 [==============================] - ETA: 0s - loss: 0.1678 - accuracy: 0.9358
Epoch 2: loss improved from 0.28393 to 0.16783, saving model to BiLSTM-model2-weights-improvement-02-0.1678-bigger.keras
716/716 [==============================] - 150s 209ms/step - loss: 0.1678 - accuracy: 0.9358 - val_loss: 0.2306 - val_accuracy: 0.9081
Epoch 3/10
716/716 [==============================] - ETA: 0s - loss: 0.1267 - accuracy: 0.9530
Epoch 3: loss improved from 0.16783 to 0.12671, saving model to BiLSTM-model2-weights-improvement-03-0.1267-bigger.keras
716/716 [==============================] - 152s 211ms/step - loss: 0.1267 - accurac



---

## Test the model and metrics

In [ ]:
model_bi_loss = Sequential()
model_bi_loss.add(Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=32))
model_bi_loss.add(LSTM(64, input_shape=(1,32), return_sequences=True))
model_bi_loss.add(Dropout(0.2))
model_bi_loss.add(LSTM(64))
model_bi_loss.add(Dropout(0.2))
model_bi_loss.add(Dense(1, activation='sigmoid'))
model_bi_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bi_loss.summary()

model_bi_val_loss = Sequential()
model_bi_val_loss.add(Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=32))
model_bi_val_loss.add(LSTM(64, input_shape=(1,32), return_sequences=True))
model_bi_val_loss.add(Dropout(0.2))
model_bi_val_loss.add(LSTM(64))
model_bi_val_loss.add(Dropout(0.2))
model_bi_val_loss.add(Dense(1, activation='sigmoid'))
model_bi_val_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bi_val_loss.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 32)          583808    
                                                                 
 lstm_24 (LSTM)              (None, None, 64)          24832     
                                                                 
 dropout_24 (Dropout)        (None, None, 64)          0         
                                                                 
 lstm_25 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_25 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 641729 (2.45 MB)
Trainable params: 6417

In [ ]:
#least training loss - Epoch 9
filename_loss = "/content/BiLSTM-model2-weights-improvement-09-0.0337-bigger.keras"
model_bi_loss.load_weights(filename_loss)
model_bi_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['lstm_24/lstm_cell/kernel:0', 'lstm_24/lstm_cell/recurrent_kernel:0', 'lstm_24/lstm_cell/bias:0']

In [ ]:
#least validation loss - Epoch 5
filename_val_loss = "/content/BiLSTM-model2-weights-improvement-05-0.0791-bigger.keras"
model_bi_val_loss.load_weights(filename_val_loss)
model_bi_val_loss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['lstm_26/lstm_cell/kernel:0', 'lstm_26/lstm_cell/recurrent_kernel:0', 'lstm_26/lstm_cell/bias:0']

In [ ]:
y_predict_bi_loss = model_bi.predict(x_test_vp)
y_predict_bi_val_loss = model_bi_val_loss.predict(x_test_vp)

448/448 [==============================] - 6s 12ms/step


In [ ]:
y_predict_bi_loss = y_predict_bi_loss.round()
y_predict_bi_val_loss = y_predict_bi_val_loss.round()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
#y_test is label
#y_predict is prediction
print("Least training loss:\n")
print("Accuracy:", accuracy_score(y_test, y_predict_bi_loss))
print( classification_report(y_test, y_predict_bi_loss))
print("\nLeast validation loss:\n")
print("Accuracy:", accuracy_score(y_test, y_predict_bi_val_loss))
print( classification_report(y_test, y_predict_val_loss))

Least training loss:

Accuracy: 0.896968426934898
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      6998
           1       0.89      0.92      0.90      7318

    accuracy                           0.90     14316
   macro avg       0.90      0.90      0.90     14316
weighted avg       0.90      0.90      0.90     14316


Least validation loss:

Accuracy: 0.5007683710533669
              precision    recall  f1-score   support

           0       0.92      0.82      0.87      6998
           1       0.84      0.93      0.89      7318

    accuracy                           0.88     14316
   macro avg       0.88      0.88      0.88     14316
weighted avg       0.88      0.88      0.88     14316



# Member Group 13

## **Members Group 13**
```
Jessada Peetapa	     6422770915	 6422770915@g.siit.tu.ac.th
Matas Thanamee	      6422771251	 6422771251@g.siit.tu.ac.th
Woramate Simrum	     6422771400	 6422771400@g.siit.tu.ac.th
Napat Khowyabud	     6422772051	 6422772051@g.siit.tu.ac.th
Pavida Pipatanagovit    6422772069	 6422772069@g.siit.tu.ac.th
```